In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
datapath = '/content/drive/MyDrive/datasets/MLQA_V1.zip'
import zipfile
extract_path = '/content/dataset'
with zipfile.ZipFile(datapath, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
import os

In [ ]:
data = extract_path + '/MLQA_V1'
print(os.listdir(data))

['dev', 'LICENSE', 'test']


In [ ]:
data = extract_path + '/MLQA_V1/test'
print(os.listdir(data))

['test-context-de-question-vi.json', 'test-context-en-question-ar.json', 'test-context-ar-question-en.json', 'test-context-zh-question-hi.json', 'test-context-en-question-hi.json', 'test-context-vi-question-de.json', 'test-context-vi-question-hi.json', 'test-context-vi-question-vi.json', 'test-context-es-question-en.json', 'test-context-es-question-de.json', 'test-context-es-question-hi.json', 'test-context-hi-question-vi.json', 'test-context-vi-question-zh.json', 'test-context-de-question-hi.json', 'test-context-ar-question-es.json', 'test-context-es-question-es.json', 'test-context-hi-question-de.json', 'test-context-en-question-es.json', 'test-context-de-question-ar.json', 'test-context-es-question-ar.json', 'test-context-en-question-vi.json', 'test-context-de-question-en.json', 'test-context-ar-question-zh.json', 'test-context-ar-question-hi.json', 'test-context-es-question-zh.json', 'test-context-de-question-es.json', 'test-context-hi-question-ar.json', 'test-context-zh-question-a

In [ ]:
import json

In [ ]:
# Getting English Text and English Context json file
data = extract_path + '/MLQA_V1/test/test-context-en-question-en.json'
with open(data, 'r') as f:
    dataset = json.load(f)

In [ ]:
dataset2 = json.dumps(dataset, indent=4)  # just for printing purpose dump the dataset into dataset2

In [ ]:
print(type(dataset2))

<class 'str'>


In [ ]:
print(dataset2)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(dataset['data'][0])

{'title': 'Area 51', 'paragraphs': [{'context': 'In 1994, five unnamed civilian contractors and the widows of contractors Walter Kasza and Robert Frost sued the USAF and the United States Environmental Protection Agency. Their suit, in which they were represented by George Washington University law professor Jonathan Turley, alleged they had been present when large quantities of unknown chemicals had been burned in open pits and trenches at Groom. Biopsies taken from the complainants were analyzed by Rutgers University biochemists, who found high levels of dioxin, dibenzofuran, and trichloroethylene in their body fat. The complainants alleged they had sustained skin, liver, and respiratory injuries due to their work at Groom, and that this had contributed to the deaths of Frost and Kasza. The suit sought compensation for the injuries they had sustained, claiming the USAF had illegally handled toxic materials, and that the EPA had failed in its duty to enforce the Resource Conservation 

In [ ]:
print(len(dataset['data']))  # 5010 different titles (stories)

5010


In [ ]:
print(len(dataset['data'][0]['paragraphs']))

8


In [ ]:
# For tokenization purpose we need to flatten the dataset.
Flat_data = {
    "id":[],
    "context":[],
    "questions":[],
    "answers":[],
}

In [ ]:
for article in dataset['data']:
  for paragraph in article['paragraphs']:
    context = paragraph['context']

    for qa in paragraph['qas']:
      # Each question has context
      Flat_data['id'].append(qa['id'])
      Flat_data['context'].append(context)
      Flat_data['questions'].append(qa['question'])
      Flat_data['answers'].append(qa['answers'])

In [ ]:
print(len(Flat_data['questions']))
print(len(Flat_data['id']))
print(len(Flat_data['answers']))
print(len(Flat_data['context']))

11590
11590
11590
11590


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset, Features, Value, Sequence

In [ ]:
model = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess(data):
  tokenized = tokenizer(
      data['context'],
      data['questions'],
      truncation = True,
      max_length = 512,
      stride = 128,
      return_offsets_mapping = True,
      return_overflowing_tokens = True,
      padding = "max_length",

  )

  sample_map = tokenized.pop("overflow_to_sample_mapping")
  offset_map = tokenized.pop("offset_mapping")

  start_pos = []
  end_pos = []

  for i,offsets in enumerate(offset_map):
    sample_idx = sample_map[i]
    answer = data["answers"][sample_idx][0] # Assuming there's always at least one answer and we take the first one
    start_idx = answer["answer_start"]
    end_idx = start_idx + len(answer["text"])

    seq_ids = tokenized.sequence_ids(i)

    # Find the start and end of the context (segment_id = 1)
    try:
        ctx_start = seq_ids.index(1)
        ctx_end = len(seq_ids) - 1 - seq_ids[::-1].index(1)
    except ValueError: # Handle cases where segment_id 1 (context) might not be found
        ctx_start = 0
        ctx_end = 0

    if offsets[ctx_start][0] > start_idx or offsets[ctx_end][1] < end_idx:
      start_pos.append(0)
      end_pos.append(0)
      continue

    token_start = 0
    token_end = 0

    for idx in range(ctx_start, ctx_end + 1):
      if offsets[idx][0] <= start_idx < offsets[idx][1]:
        token_start = idx
      if offsets[idx][0] < end_idx <= offsets[idx][1]:
        token_end = idx

    start_pos.append(token_start)
    end_pos.append(token_end)
  tokenized["start_positions"] = start_pos
  tokenized["end_positions"] = end_pos

  return tokenized

In [ ]:
# Loading the datasets and preprocessing using Dataset module

train_dataset = Dataset.from_dict(Flat_data)
tokenized_train = train_dataset.map(preprocess,batched=True, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
tokenized_train.set_format(
    type="torch",
    columns=["input_ids", "attention_mask","token_type_ids", "start_positions", "end_positions"],
)
train_loader = DataLoader(
    tokenized_train,
    batch_size=4, # Reduced batch size to prevent OOM
    shuffle=True,
)

In [ ]:
from transformers import BertForQuestionAnswering
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
Model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
Model.to(device)

cuda


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
from torch.optim import AdamW

# Training Loop

In [ ]:
optimizer = AdamW(Model.parameters(), lr=3e-5)

for epoch in range(2):
  Model.train()

  for batch in train_loader:
    batch = {k:v.to(device) for k,v in batch.items()}
    outputs = Model(
        input_ids = batch['input_ids'],
        attention_mask = batch['attention_mask'],
        token_type_ids = batch['token_type_ids'],
        start_positions = batch['start_positions'].long(), # Explicitly cast to long
        end_positions = batch['end_positions'].long(),     # Explicitly cast to long
    )
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 3092 has 14.73 GiB memory in use. Of the allocated memory 14.48 GiB is allocated by PyTorch, and 123.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)